In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import os
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils import compute_class_weight
import matplotlib.pyplot as plt

In [15]:
# widths = []
# heights = []

# for img_name in os.listdir("Cropped/UTKFace"):
#     if img_name.endswith(".jpg"):
#         img_path = os.path.join("Cropped/UTKFace", img_name)
#         im = Image.open(img_path)
#         widths.append(im.size[0])
#         heights.append(im.size[1])

# AVG_HEIGHT = round(sum(heights)/len(heights))
# AVG_WIDTH = round(sum(widths)/len(widths))

In [16]:
# print(AVG_HEIGHT, AVG_WIDTH)

200 200


In [17]:
data = []
for filename in os.listdir("Cropped/UTKFace"):
    if filename.endswith(".jpg"):
        with Image.open(os.path.join("Cropped/UTKFace", filename)):
            row_data = [os.path.join("Cropped/UTKFace", filename)]
            filename = filename.split("_")
            if len(filename) > 3:
                # TODO REPLACE INT WITH NAME
                row_data.append(filename[2]) # 3rd integer represents race
                data.append(row_data)

df_image = pd.DataFrame(data, columns = ["filename", "Race"])

df_image = df_image[df_image["Race"] != "4"]

train, test = train_test_split(df_image, stratify = df_image["Race"], test_size=0.2)

train = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255).flow_from_dataframe(
    dataframe = train,
    x_col = "filename",
    y_col = "Race",
    class_mode = "categorical",
    target_size = (200, 200),
    fill_mode = "nearest",
    shuffle=True
)

test = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255).flow_from_dataframe(
    dataframe = test,
    x_col = "filename",
    y_col = "Race",
    class_mode = "categorical",
    fill_mode = "nearest",
    target_size=(200, 200)
)
    
# batch = next(train)
# a = np.array(batch[0])

# img = Image.fromarray(a[0].astype('uint8'), "RGB")
# img.save(f"test.png")
# img.show()

Found 17610 validated image filenames belonging to 4 classes.
Found 4403 validated image filenames belonging to 4 classes.


In [18]:
df_image["Race"].value_counts(normalize=True)

Race
0    0.457820
1    0.205606
3    0.180575
2    0.155999
Name: proportion, dtype: float64

In [19]:
compute_class_weight(
    "balanced", 
    classes = np.array(["0", "1", "2", "3"]),
    y = df_image["Race"])

# compute_class_weight(
#     "balanced", 
#     classes = np.array(["0", "1"]),
#     y = df_image["Race"])
# tf.config.list_physical_devices("GPU")

array([0.54606569, 1.21591913, 1.60257717, 1.38446541])

In [31]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(200, 200, 3)))
model.add(keras.layers.Dense(512, activation = "relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(300, activation = "relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(128, activation = "relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(4, activation = "softmax"))
model.compile(loss = "CategoricalCrossentropy", metrics = ["accuracy"], optimizer=keras.optimizers.Adam())
model.fit(train,
          class_weight={0:0.5476668 , 1:1.2282251, 2:1.56114055, 3:1.39017879},
          validation_data = test,
          batch_size = 16,
          epochs = 20)

Epoch 1/20
551/551 [==============================] - 15s 27ms/step - loss: 4.8525 - accuracy: 0.2140 - val_loss: 1.3869 - val_accuracy: 0.1812
Epoch 2/20
551/551 [==============================] - 14s 26ms/step - loss: 1.3935 - accuracy: 0.2140 - val_loss: 1.3835 - val_accuracy: 0.2055
Epoch 3/20
 91/551 [===>..........................] - ETA: 9s - loss: 1.3949 - accuracy: 0.2128

KeyboardInterrupt: 

In [22]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, (6, 6), input_shape=(200, 200, 3), activation = "relu"))
model.add(keras.layers.Conv2D(32, (6, 6), activation = "relu"))
model.add(keras.layers.MaxPool2D(2, 2))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation = "relu"))
model.add(keras.layers.BatchNormalization()),
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(4, activation = "softmax"))


model.compile(loss = "CategoricalCrossentropy", metrics = ["accuracy"], optimizer=keras.optimizers.Adam())
model.fit(train,
          validation_data = test,
          class_weight={0:0.5476668, 1:1.2282251 , 2:1.56114055, 3:1.39017879},
          batch_size = 16,
          epochs = 15)

Epoch 1/15
551/551 [==============================] - 53s 91ms/step - loss: 0.8425 - accuracy: 0.6569 - val_loss: 1.2092 - val_accuracy: 0.4731
Epoch 2/15
551/551 [==============================] - 47s 84ms/step - loss: 0.6435 - accuracy: 0.7544 - val_loss: 1.7191 - val_accuracy: 0.4817
Epoch 3/15
551/551 [==============================] - 52s 94ms/step - loss: 0.5706 - accuracy: 0.7829 - val_loss: 0.6433 - val_accuracy: 0.7663
Epoch 4/15
551/551 [==============================] - 47s 86ms/step - loss: 0.5351 - accuracy: 0.7949 - val_loss: 0.5495 - val_accuracy: 0.8006
Epoch 5/15
551/551 [==============================] - 46s 83ms/step - loss: 0.5178 - accuracy: 0.7998 - val_loss: 0.5730 - val_accuracy: 0.8006
Epoch 6/15
551/551 [==============================] - 50s 90ms/step - loss: 0.4427 - accuracy: 0.8296 - val_loss: 0.5424 - val_accuracy: 0.8076
Epoch 7/15
551/551 [==============================] - 52s 94ms/step - loss: 0.3777 - accuracy: 0.8511 - val_loss: 1.0092 - val_accuracy:

In [23]:
model.save("cnn_model")

INFO:tensorflow:Assets written to: cnn_model\assets


INFO:tensorflow:Assets written to: cnn_model\assets
